# PDF 文書の内容にもとづいて回答するボットの作り方を理解する
## はじめに
生成 AI や大規模言語モデル （LLM）は、膨大な量のテキストデータのトレーニングを受けており、テキストを生成し、言語を翻訳し、さまざまな種類のクリエイティブ コンテンツを作成し、有益な方法で質問に答えることができます。ただし、LLM には、企業で使用する場合の問題点がいくつかあります。
その一つに、生成 AI は誤った情報を含むテキストを生成したり、誤った情報を翻訳したりする可能性があります。企業における生成 AI 利用では、これが問題となることがあります。

そこで生成 AI をエンジンとして利用しながら、企業内の信頼のおける情報の中から適切な回答してほしいというユースケースがあります。

これを実現する大きな処理の流れは次のようになっています。

1.   PDF ファイルを読み込む
2.   ページごとでもよいのですが、もう少し細かい単位にテキストを分割する
3.   分割したテキスト情報を PaLM API を利用してベクトル化 / エンべディング
4.   ベクトル情報とテキスト情報の関連を保持する
5.   ユーザーの問い合わせ内容をベクトル化 / エンべディングする
6.   5. のベクトルと最も類似度の高いベクトルを複数個取得する
7.   6. のベクトルを生成元のテキスト情報を取得する
8.   複数個のテキスト情報をコンテキスト情報として与え、この情報の中から質問の回答を作成するように LLM に問い合わせをする
9.   PDF 文書の内容にもとづく回答が得られます

これを実現するためにエンべディング、ベクトル検索やコンテンツの保持といった技術要素が必要となりますが、Google Cloud では Generetive AI App Builder - Enterprise Search というソリューションを提供しているため、個々の技術要素をバラバラに組み合わせるのではなく、Google の強みである検索技術をとりこんだソリューションを活用して簡単にソリューションを構築することが可能です。

ここでは、生成 AI 利用時のフレームワークである LangChain を利用して、Enterprise Search を利用して企業の信頼するデータソースから情報を取得し、その情報をもとにChatbot型で質問に回答する流れを確認します。

## Enterprise Search のセットアップ

はじめに、Cloud Storage にバケットを作成し、回答させたいコンテンツを含む PDF ファイルをアップロードしておきます。
このハンズオンでは、次のファイルをアップロードする前提で記載しています。
 - [Alphabet 2022 10K annual report](https://abc.xyz/assets/9a/bd/838c917c4b4ab21f94e84c3c2c65/goog-10-k-q4-2022.pdf)
 - [Google security overview](https://cloud.google.com/static/docs/security/overview/resources/google_security_wp.pdf)
 - [Google infrastructure security design overview](https://cloud.google.com/static/docs/security/infrastructure/design/resources/google_infrastructure_whitepaper_fa.pdf)

次に、リンク先の手順に従って Enterprise Search をセットアップします。

[Create and preview an unstructured data search ap](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_and_preview_an_unstructured_data_search_app)

パラメーターは以下を指定していることを確認してください。
- アプリケーションの種類: 検索
- インデックスするデータの種類: 非構造化データ( PDF や HTML )

インデックスの作成にしばらく時間がかかります。Gen App Builder のプレビュー機能で検索ができるか確認することが可能です。

最後に、Gen App Builder のアプリ一覧画面において、作成したアプリケーションの ID をメモしておきます。

## 環境セットアップ

python のバージョンを確認します。最新の LangChain は `Requires-Python >=3.8.1,<4.0`が前提となっています。

In [ ]:
import sys
print(sys.version)

前提パッケージを導入します。

In [ ]:
# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform google-cloud-discoveryengine langchain==0.0.236  --upgrade --user

※ 注意1

* Colab の場合、上記のログに"RESTART RUNTIME"ボタンをが表示された場合、ボタンを押してカーネルをリスタートします。
* Vertex AI Workbench の場合、メニューよりカーネルのリスタートを実行します。


続いて、Google Cloud でプロジェクトを作成し Vertex AI API を有効化します。また、このコードを実行するユーザーに`Vertex AI ユーザー`のロールを付与します。


※ 注意2
* Colab の場合、以下をコメントアウトして Vertex AI API のユーザー権限をもつアカウントでログインします。

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

環境変数などを定義します。 Google Cloud のプロジェクト ID、Enterprise Search の ID を指定してください。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
SEARCH_ENGINE_ID = "<search_engine_id>"  # @param {type:"string"}
REGION = "us-central1"

Vertex AI と LangChain のライブラリーの導入を確認します。 LangChain v0.0.208 で動作確認しています。

In [ ]:
import langchain
from google.cloud import aiplatform

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

## Vertex AI PaLM API の準備

LangChain を利用して PaLM API のText、Chat、Embeddings モデルを取得します。

In [ ]:
from langchain.llms import VertexAI

# Text model instance integrated with LangChain
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [ ]:
from langchain.retrievers import GoogleCloudEnterpriseSearchRetriever

# Enterprise Search retriever
retriever = GoogleCloudEnterpriseSearchRetriever(project_id=PROJECT_ID, search_engine_id=SEARCH_ENGINE_ID, get_extractive_answers=False)

## Enterprise Search に格納した PDF の内容で Q/A をする

ユーザーが質問した質問に対して、Enterprise Search の検索で取得した類似のテキスト情報の中から回答を出すようにします。この仕組みを簡単に実現するフレームワークとして LangChain の RetrievalQA を利用します。

In [ ]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

ここで質問を定義してみましょう。

In [ ]:
#query = input("Enter query:")
query = "How does Google Cloud protect customer data?" # @param {type:"string"}


PaLM API for text に直接問い合わせをした場合はどのような回答になるでしょう？

In [ ]:
llm(query)

PDF 情報からの回答はどのようになるでしょう？

In [ ]:
response = qa({"query": query})
response["result"]

回答内容を作成したソースを確認します。

In [ ]:
response["source_documents"]

どうして PDF の内容をもとに回答しているかを理解するには、上で利用した LangChain の RetrievalQA の `stuff` タイプのソースコードを参照するのが分かりやすいです。

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/question_answering/stuff_prompt.py

```
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
```
訳
```
次のコンテキストをもとに、最後の質問に答えてください。もし答えが分からない場合は、分からないと答えてください。勝手に答えをでっち上げないでください。
```

以上です。ありがとうございました。

## 参考情報
- [Question Answering Over Documents](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gen-app-builder/retrieval-augmented-generation/examples/question_answering.ipynb)
- [LangChain](https://python.langchain.com/docs/get_started/introduction.html)
- [PaLM API on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)
